In [7]:
import torch
import onnx
import os
from transformers import WavLMModel

#wavlm_model_version = "microsoft/wavlm-large"
#output_file = "wavlm_large_layer9.onnx"
#wavlm_model_version = "microsoft/wavlm-base-plus"
#output_file = "wavlm_base_plus_layer9.onnx"
wavlm_model_version = "microsoft/wavlm-base"
output_file = "wavlm_base_layer9.onnx"

# Load WavLM model
print("Loading WavLM model...")
model = WavLMModel.from_pretrained(wavlm_model_version)

# Create a truncated model that only outputs the 9th layer
class WavLMLayer9(torch.nn.Module):
    def __init__(self, wavlm_model):
        super().__init__()
        self.wavlm = wavlm_model

    def forward(self, input_values):
        outputs = self.wavlm(input_values, output_hidden_states=True)
        # Extract 9th layer (index 9 in the hidden_states tuple)
        ninth_layer = outputs.hidden_states[9]
        return ninth_layer

# Create the truncated model
print("Creating truncated model...")
truncated_model = WavLMLayer9(model)

# Set the model to evaluation mode
truncated_model.eval()

# Export to ONNX for browser use
print("Exporting to ONNX with opset version 14...")
dummy_input = torch.randn(1, 16000)  # 1 second of audio at 16kHz
torch.onnx.export(
    truncated_model,
    dummy_input,
    output_file,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size", 1: "audio_length"},
                  "output": {0: "batch_size", 1: "sequence_length"}},
    opset_version=14,  # Updated to version 14
    do_constant_folding=True,
    verbose=False
)

# Verify the model
print("Verifying ONNX model...")
onnx_model = onnx.load(output_file)
onnx.checker.check_model(onnx_model)

print("Model exported successfully to " + output_file)
print(f"Model size: {os.path.getsize(output_file) / (1024 * 1024):.2f} MB")


Loading WavLM model...


config.json:   0%|          | 0.00/2.24k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Creating truncated model...
Exporting to ONNX with opset version 14...


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Verifying ONNX model...
Model exported successfully to wavlm_base_layer9.onnx
Model size: 279.34 MB


In [8]:
# Quantize the model to reduce file size

# Install ONNX Runtime with quantization support
!pip install onnxruntime onnx

# The quantization tools are in a separate package
!pip install onnxruntime-extensions

try:
    from onnxruntime.quantization import quantize_dynamic, QuantType
    print("Quantizing model...")
    quantize_dynamic(
        output_file,
        'quantized_'+output_file,
        weight_type=QuantType.QUInt8
    )
    print(f"Quantized model size: {os.path.getsize('quantized_'+output_file) / (1024 * 1024):.2f} MB")
except ImportError:
    print("Skipping quantization - onnxruntime.quantization not available")

Quantizing model...


Quantized model size: 70.68 MB


In [9]:
# Check if file exists
if os.path.exists(output_file):
    from google.colab import files

    print("Downloading model file...")
    files.download(output_file)

    # Also download quantized version if it exists
    if os.path.exists('quantized_'+output_file):
        print("Downloading quantized model file...")
        files.download('quantized_'+output_file)
else:
    print("Model file not found!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>